In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Define date ranges for training and evaluation
TRAIN_START = '2021-07-15'
TRAIN_END = '2021-11-14'
EVAL_START = '2021-11-15'
EVAL_END = '2022-01-14'

print(f"Training range: {TRAIN_START} → {TRAIN_END}")
print(f"Evaluation range: {EVAL_START} → {EVAL_END}")
print(f"Total data range: {TRAIN_START} → {EVAL_END}")

Training range: 2021-07-15 → 2021-11-14
Evaluation range: 2021-11-15 → 2022-01-14
Total data range: 2021-07-15 → 2022-01-14


In [2]:
# 1. Load deaths_malaysia.csv (Target variable: deaths_new)
print("Loading deaths_malaysia.csv...")
deaths_df = pd.read_csv('deaths_malaysia.csv')
deaths_df['date'] = pd.to_datetime(deaths_df['date'])

# Filter date range
deaths_df = deaths_df[(deaths_df['date'] >= TRAIN_START) & (deaths_df['date'] <= EVAL_END)]
deaths_df = deaths_df[['date', 'deaths_new']].copy()

print(f"Deaths data shape: {deaths_df.shape}")
print(f"Date range: {deaths_df['date'].min()} to {deaths_df['date'].max()}")
print("Sample data:")
print(deaths_df.head())

Loading deaths_malaysia.csv...
Deaths data shape: (184, 2)
Date range: 2021-07-15 00:00:00 to 2022-01-14 00:00:00
Sample data:
          date  deaths_new
485 2021-07-15         110
486 2021-07-16         115
487 2021-07-17         138
488 2021-07-18         153
489 2021-07-19         129


In [3]:
# 2. Load cases_malaysia.csv (Features: cases_new, cases_active)
print("Loading cases_malaysia.csv...")
cases_df = pd.read_csv('cases_malaysia.csv')
cases_df['date'] = pd.to_datetime(cases_df['date'])

# Filter date range and select relevant columns
cases_df = cases_df[(cases_df['date'] >= TRAIN_START) & (cases_df['date'] <= EVAL_END)]
cases_df = cases_df[['date', 'cases_new', 'cases_active']].copy()

print(f"Cases data shape: {cases_df.shape}")
print("Sample data:")
print(cases_df.head())

Loading cases_malaysia.csv...
Cases data shape: (184, 3)
Sample data:
          date  cases_new  cases_active
537 2021-07-15      13215        108370
538 2021-07-16      12541        114054
539 2021-07-17      12528        119815
540 2021-07-18      10710        124594
541 2021-07-19      10972        128998


In [4]:
# 3. Load icu.csv (Features: icu_covid, vent_covid - ICU and ventilator usage)
print("Loading icu.csv...")
icu_df = pd.read_csv('icu.csv')
icu_df['date'] = pd.to_datetime(icu_df['date'])

# Filter date range
icu_df = icu_df[(icu_df['date'] >= TRAIN_START) & (icu_df['date'] <= EVAL_END)]

# Aggregate by date (sum across all states) and select relevant columns
# icu_covid = ICU COVID patients, vent_covid = COVID patients on ventilators
icu_agg = icu_df.groupby('date').agg({
    'icu_covid': 'sum',
    'vent_covid': 'sum'
}).reset_index()

# Rename columns for clarity
icu_agg.columns = ['date', 'icu', 'icu_vent']

print(f"ICU data shape: {icu_agg.shape}")
print("Sample data:")
print(icu_agg.head())

Loading icu.csv...
ICU data shape: (184, 3)
Sample data:
        date   icu  icu_vent
0 2021-07-15  1206       716
1 2021-07-16  1270       709
2 2021-07-17  1312       729
3 2021-07-18  1317       734
4 2021-07-19  1330       737


In [5]:
# 4. Load hospital.csv (Features: hosp_covid, admitted_covid - hospitalization data)
print("Loading hospital.csv...")
hosp_df = pd.read_csv('hospital.csv')
hosp_df['date'] = pd.to_datetime(hosp_df['date'])

# Filter date range
hosp_df = hosp_df[(hosp_df['date'] >= TRAIN_START) & (hosp_df['date'] <= EVAL_END)]

# Aggregate by date (sum across all states)
# hosp_covid = total COVID hospitalizations, admitted_covid = new COVID admissions
hosp_agg = hosp_df.groupby('date').agg({
    'hosp_covid': 'sum',
    'admitted_covid': 'sum'
}).reset_index()

# Rename columns for clarity
hosp_agg.columns = ['date', 'hosp', 'hosp_new']

print(f"Hospital data shape: {hosp_agg.shape}")
print("Sample data:")
print(hosp_agg.head())

Loading hospital.csv...
Hospital data shape: (184, 3)
Sample data:
        date   hosp  hosp_new
0 2021-07-15   9606      1626
1 2021-07-16  10000      1489
2 2021-07-17  10080      1768
3 2021-07-18  10378      1719
4 2021-07-19  11375      1620


In [6]:
# 5. Load tests_malaysia.csv (Features: rtk-ag, pcr - testing intensity)
print("Loading tests_malaysia.csv...")
tests_df = pd.read_csv('tests_malaysia.csv')
tests_df['date'] = pd.to_datetime(tests_df['date'])

# Filter date range and select relevant columns
tests_df = tests_df[(tests_df['date'] >= TRAIN_START) & (tests_df['date'] <= EVAL_END)]
tests_df = tests_df[['date', 'rtk-ag', 'pcr']].copy()

print(f"Tests data shape: {tests_df.shape}")
print("Sample data:")
print(tests_df.head())

Loading tests_malaysia.csv...
Tests data shape: (184, 3)
Sample data:
          date  rtk-ag    pcr
538 2021-07-15   77701  74255
539 2021-07-16   68659  75090
540 2021-07-17   59602  75689
541 2021-07-18   50242  59769
542 2021-07-19   73269  60676


In [7]:
# 6. Load vax_malaysia.csv (Vaccination features)
print("Loading vax_malaysia.csv...")
vax_df = pd.read_csv('vax_malaysia.csv')
vax_df['date'] = pd.to_datetime(vax_df['date'])

# Filter date range and select relevant columns
vax_df = vax_df[(vax_df['date'] >= TRAIN_START) & (vax_df['date'] <= EVAL_END)]

# Select key vaccination columns
vax_columns = ['date', 'daily_partial', 'daily_full', 'daily_booster', 'cumul_full']
vax_df = vax_df[vax_columns].copy()

print(f"Vaccination data shape: {vax_df.shape}")
print("Sample data:")
print(vax_df.head())

Loading vax_malaysia.csv...
Vaccination data shape: (184, 5)
Sample data:
          date  daily_partial  daily_full  daily_booster  cumul_full
141 2021-07-15         317950      161424              0     4269859
142 2021-07-16         284002      134328              0     4404187
143 2021-07-17         264820      132945              0     4537132
144 2021-07-18         252620      104252              0     4641384
145 2021-07-19         288969      154686              0     4796070


In [8]:
# 7. Merge all datasets by date
print("Merging all datasets...")

# Start with deaths as the base (target variable)
merged_df = deaths_df.copy()

# Merge each dataset
datasets_to_merge = [
    (cases_df, 'cases'),
    (icu_agg, 'icu'),
    (hosp_agg, 'hospital'),
    (tests_df, 'tests'),
    (vax_df, 'vaccination')
]

for df, name in datasets_to_merge:
    print(f"  Merging {name} data...")
    merged_df = pd.merge(merged_df, df, on='date', how='inner')
    print(f"  Shape after merging {name}: {merged_df.shape}")

print(f"\nFinal merged dataset shape: {merged_df.shape}")
print(f"Date range: {merged_df['date'].min()} to {merged_df['date'].max()}")
print("\nColumn names:")
print(list(merged_df.columns))

Merging all datasets...
  Merging cases data...
  Shape after merging cases: (184, 4)
  Merging icu data...
  Shape after merging icu: (184, 6)
  Merging hospital data...
  Shape after merging hospital: (184, 8)
  Merging tests data...
  Shape after merging tests: (184, 10)
  Merging vaccination data...
  Shape after merging vaccination: (184, 14)

Final merged dataset shape: (184, 14)
Date range: 2021-07-15 00:00:00 to 2022-01-14 00:00:00

Column names:
['date', 'deaths_new', 'cases_new', 'cases_active', 'icu', 'icu_vent', 'hosp', 'hosp_new', 'rtk-ag', 'pcr', 'daily_partial', 'daily_full', 'daily_booster', 'cumul_full']


In [9]:
# 8. Create lagged features for better prediction
print("Creating lagged features...")

# Sort by date to ensure proper lag calculation
merged_df = merged_df.sort_values('date').reset_index(drop=True)

# Define features that should have lags
lag_features = {
    'cases_new': [7, 14],  # Cases lagged by 1 and 2 weeks
    'icu': [7],            # ICU usage lagged by 1 week
    'cumul_full': [14]     # Vaccination effect takes ~2 weeks
}

# Create lagged features
for feature, lags in lag_features.items():
    for lag in lags:
        col_name = f"{feature}_lag{lag}"
        merged_df[col_name] = merged_df[feature].shift(lag)
        print(f"Created {col_name}")

# Create 7-day rolling averages for daily metrics to smooth noise
rolling_features = ['deaths_new', 'cases_new', 'hosp_new', 'daily_full']

for feature in rolling_features:
    col_name = f"{feature}_7d_avg"
    merged_df[col_name] = merged_df[feature].rolling(window=7, min_periods=1).mean()
    print(f"Created {col_name}")

print(f"\nDataset shape after feature engineering: {merged_df.shape}")
print("New columns added:")
new_cols = [col for col in merged_df.columns if '_lag' in col or '_7d_avg' in col]
print(new_cols)

Creating lagged features...
Created cases_new_lag7
Created cases_new_lag14
Created icu_lag7
Created cumul_full_lag14
Created deaths_new_7d_avg
Created cases_new_7d_avg
Created hosp_new_7d_avg
Created daily_full_7d_avg

Dataset shape after feature engineering: (184, 22)
New columns added:
['cases_new_lag7', 'cases_new_lag14', 'icu_lag7', 'cumul_full_lag14', 'deaths_new_7d_avg', 'cases_new_7d_avg', 'hosp_new_7d_avg', 'daily_full_7d_avg']


In [10]:
# 9. Split data into training and evaluation sets
print("Splitting data into train/eval sets...")

# Training set: 15 July 2021 → 14 November 2021
train_df = merged_df[(merged_df['date'] >= TRAIN_START) & (merged_df['date'] <= TRAIN_END)].copy()

# Evaluation set: 15 November 2021 → 14 January 2022  
eval_df = merged_df[(merged_df['date'] >= EVAL_START) & (merged_df['date'] <= EVAL_END)].copy()

print(f"Training set shape: {train_df.shape}")
print(f"Training date range: {train_df['date'].min()} to {train_df['date'].max()}")
print(f"Training period: {len(train_df)} days")

print(f"\nEvaluation set shape: {eval_df.shape}")
print(f"Evaluation date range: {eval_df['date'].min()} to {eval_df['date'].max()}")
print(f"Evaluation period: {len(eval_df)} days")

print(f"\nTotal dataset: {len(merged_df)} days")
print(f"Train ratio: {len(train_df)/len(merged_df):.1%}")
print(f"Eval ratio: {len(eval_df)/len(merged_df):.1%}")

Splitting data into train/eval sets...
Training set shape: (123, 22)
Training date range: 2021-07-15 00:00:00 to 2021-11-14 00:00:00
Training period: 123 days

Evaluation set shape: (61, 22)
Evaluation date range: 2021-11-15 00:00:00 to 2022-01-14 00:00:00
Evaluation period: 61 days

Total dataset: 184 days
Train ratio: 66.8%
Eval ratio: 33.2%


In [11]:
# 10. Data quality check and summary
print("=== DATA QUALITY SUMMARY ===")
print(f"Full dataset shape: {merged_df.shape}")
print(f"Missing values per column:")
print(merged_df.isnull().sum())

print("\n=== FEATURE OVERVIEW ===")
feature_groups = {
    'Target': ['deaths_new'],
    'Cases': ['cases_new', 'cases_active', 'cases_new_lag7', 'cases_new_lag14'],
    'Healthcare Load': ['icu', 'icu_vent', 'icu_lag7', 'hosp', 'hosp_new'],
    'Testing': ['rtk-ag', 'pcr'],
    'Vaccination': ['daily_partial', 'daily_full', 'daily_booster', 'cumul_full', 'cumul_full_lag14'],
    'Rolling Averages': ['deaths_new_7d_avg', 'cases_new_7d_avg', 'hosp_new_7d_avg', 'daily_full_7d_avg']
}

for group, features in feature_groups.items():
    print(f"\n{group}:")
    available_features = [f for f in features if f in merged_df.columns]
    for f in available_features:
        print(f"  - {f}")

print("\n=== SAMPLE DATA ===")
print(merged_df[['date', 'deaths_new', 'cases_new', 'icu', 'hosp', 'cumul_full']].head(10))

=== DATA QUALITY SUMMARY ===
Full dataset shape: (184, 22)
Missing values per column:
date                  0
deaths_new            0
cases_new             0
cases_active          0
icu                   0
icu_vent              0
hosp                  0
hosp_new              0
rtk-ag                0
pcr                   0
daily_partial         0
daily_full            0
daily_booster         0
cumul_full            0
cases_new_lag7        7
cases_new_lag14      14
icu_lag7              7
cumul_full_lag14     14
deaths_new_7d_avg     0
cases_new_7d_avg      0
hosp_new_7d_avg       0
daily_full_7d_avg     0
dtype: int64

=== FEATURE OVERVIEW ===

Target:
  - deaths_new

Cases:
  - cases_new
  - cases_active
  - cases_new_lag7
  - cases_new_lag14

Healthcare Load:
  - icu
  - icu_vent
  - icu_lag7
  - hosp
  - hosp_new

Testing:
  - rtk-ag
  - pcr

Vaccination:
  - daily_partial
  - daily_full
  - daily_booster
  - cumul_full
  - cumul_full_lag14

Rolling Averages:
  - deaths_new_7d_avg


In [12]:
# 11. Save processed datasets
print("Saving processed datasets...")

# Save full merged dataset
merged_df.to_csv('processed_covid_data.csv', index=False)
print("✅ Saved: processed_covid_data.csv")

# Save training set
train_df.to_csv('train_data.csv', index=False)
print("✅ Saved: train_data.csv")

# Save evaluation set  
eval_df.to_csv('eval_data.csv', index=False)
print("✅ Saved: eval_data.csv")

print("\n=== SUMMARY ===")
print(f"📊 Total features: {len(merged_df.columns) - 1}")  # -1 for date column
print(f"🎯 Target variable: deaths_new")
print(f"📅 Date range: {TRAIN_START} → {EVAL_END}")
print(f"🚂 Training: {len(train_df)} days ({TRAIN_START} → {TRAIN_END})")
print(f"🔍 Evaluation: {len(eval_df)} days ({EVAL_START} → {EVAL_END})")
print(f"\n✨ Ready for machine learning model training!")

print(f"\n📝 Note: Lagged features have some missing values at the beginning:")
print(f"   - 7-day lags: {7} missing values")
print(f"   - 14-day lags: {14} missing values")
print(f"   These can be handled during model training (drop or impute).")

Saving processed datasets...
✅ Saved: processed_covid_data.csv
✅ Saved: train_data.csv
✅ Saved: eval_data.csv

=== SUMMARY ===
📊 Total features: 21
🎯 Target variable: deaths_new
📅 Date range: 2021-07-15 → 2022-01-14
🚂 Training: 123 days (2021-07-15 → 2021-11-14)
🔍 Evaluation: 61 days (2021-11-15 → 2022-01-14)

✨ Ready for machine learning model training!

📝 Note: Lagged features have some missing values at the beginning:
   - 7-day lags: 7 missing values
   - 14-day lags: 14 missing values
   These can be handled during model training (drop or impute).
